In [ ]:
import duckdb
import pandas as pd
import geopandas as gpd
from osgeo import ogr, osr
import subprocess
import shapely.wkb

In [ ]:
db = duckdb.connect("duckdb/overturemaps.db")

In [ ]:
db.sql("""
INSTALL spatial;
INSTALL httpfs;
INSTALL parquet;
LOAD spatial;
LOAD httpfs;
LOAD parquet;
""")

In [ ]:
%%time
#get road data for İstanbul city by identifying a bbox
db.sql(f"""CREATE TABLE roads AS SELECT * FROM 's3://overturemaps-us-west-2/release/2024-07-22.0/theme=transportation/type=segment/*.parquet' WHERE
        subtype = 'road'
        AND bbox.xmin > 27.9564294411581 AND bbox.xmax < 29.95890758376491
        AND bbox.ymin > 40.78740852912713 AND bbox.ymax < 41.59872541544164
""")

In [ ]:
%%time
# Export the 'roads' table to Parquet
db.sql("COPY (SELECT * FROM building) TO 'roads.parquet' (FORMAT 'parquet')")

print("Export completed: roads table to roads.parquet")

In [ ]:
%%time
# Execute a SELECT query
roads = db.sql("SELECT * FROM roads").fetchdf()

# Convert the geometry column from bytearray (WKB) to shapely geometry objects
roads['geometry'] = roads['geometry'].apply(lambda x: shapely.wkb.loads(bytes(x)))
roads


In [ ]:
gdf = gpd.GeoDataFrame(roads, geometry='geometry')
gdf.crs = "EPSG:4326"
gdf

In [ ]:
gdf.to_file("roads.gpkg", layer='roads', driver="GPKG")